# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")
print(len(song_files))

72


In [5]:
filepath = song_files[0]
print(song_files)

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json', '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json', '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json', '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json', '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json', '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json', '/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json', '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json', '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json', '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json', '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json', '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAAW128F429D538-checkpoint.json', '/home/workspace/data/song_data/A/A/B/TRAABNV128F425CEE1.json', '/home/workspace/data/song_data/A/A/B/TRAABRB128F9306DD5.json', '/home/workspace/data/song_data/A/A/B/TRAABLR128F423B7E3.json', '/home/wo

In [8]:
df = pd.DataFrame(pd.read_json(filepath, lines=True, orient='columns'))
print(df.head())
print(len(df))

            artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0  
1


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
# Read the array to a list and set it to song_data
song_data = (df.values[0][7], df.values[0][8], df.values[0][0], df.values[0][9], df.values[0][5])
#song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = (df.values[0][0], df.values[0][4], df.values[0][2], df.values[0][1], df.values[0][3])
#artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data")
print(len(log_files))

In [ ]:
filepath = log_files[0]

In [ ]:
df = pd.DataFrame(pd.read_json(filepath, lines=True, orient='columns'))
df.head()
df_orig = df
df

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df['page']=='NextSong']
#df.head()
df.page.unique()

In [ ]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

In [ ]:
time_data = list(zip(t.dt.strftime('%Y-%m-%d %I:%M:%S'), t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [ ]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_data = df_orig.get(['userId', 'firstName', 'lastName', 'gender', 'level'])
#column names
user_data.columns= ['user_id', 'first_name', 'last_name', 'gender', 'level']
print(user_data)
#remove blank records
user_data_clean= user_data[user_data['user_id']!='']
#remove duplicates
user_data_duplicates_removed= user_data_clean.drop_duplicates('user_id', keep='first')
user_df = user_data_duplicates_removed
print(user_df)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
#cur.execute("SELECT s.song_id, a.artist_id FROM songs AS s LEFT JOIN artists AS a ON a.artist_id = s.artist_id \
#             WHERE a.name = ('Manowar') AND s.duration = (247.562) AND s.title = ('Shell Shock');")
# get songid and artistid from song and artist tables
try:
    for index, row in df.iterrows():
        cur.execute(song_select, (row.song, row.artist, row.length))
except psycopg2.Error as e: 
        print("Error: Inserting Rows")
        print(songplay_table_insert)
        print(songplay_data)
        print(df.iterrows())
        print(e)
try:
        results = cur.fetchone()
        print(results)
    
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None
            # insert songplay record
        start_time = pd.to_datetime(row.ts, unit='ms').strftime('%Y-%m-%d %I:%M:%S')
    
    #   insert songplay record
        print(songid, artistid)
        songplay_data = (start_time, row.userId, row.level, str(songid), str(artistid), row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
  #  print(songplay_data)
    print (e)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.